# SSI Algorithm Demo

Add related components from algotrade library 

In [ ]:
# at the beginning of the script
import gevent.monkey
gevent.monkey.patch_all()

import ujson
import time
from algotrade import redis, ssi_api, ssi_stream, handler

Proceed returned data from SSI, we will update open position and total profit if the current position is closed        
If `filledQty` is greater than zero, calculate the profit/loss from this position and sum up with `TOTAL_PROFIT`     
After that, update `OPEN_POSITION` to initialized state -> allow to open new position

Init ssi_api, ssi_stream with user's configuration    
- ssi_api is used to call api to login, open position, ...
- ssi_stream is used to get position information (filled, cancelled, ...)

In [ ]:
ALL_F1M_PRICE_TICKS = 'ALL_F1M'
OPEN_ATC = False
HANDLED_REQUESTS = []

In [ ]:
def handle_order_ok(data):
    global OPEN_POSITION, CURRENT_ORDER, TOTAL_PROFIT, HANDLED_REQUESTS
    request_id = data['uniqueID']
    print('-------------BEFORE UPDATE------------')
    print('OPEN_POSITION', OPEN_POSITION)
    print('CURRENT_ORDER', CURRENT_ORDER)
    print('--------------------------------------')
    print('HANDLED_REQUESTS: {}'.format(HANDLED_REQUESTS))
    if CURRENT_ORDER['request_id'] == request_id:
        CURRENT_ORDER['order_id'] = data['orderID']
        if data['filledQty'] > 0 and request_id not in HANDLED_REQUESTS:
            HANDLED_REQUESTS.append(request_id) # avoid same order events are returned

            # update TOTAL_PROFIT
            filled_price = data['avgPrice']
            TOTAL_PROFIT += calculate_profit(filled_price)
            print('TOTAL_PROFIT', TOTAL_PROFIT)

            # update OPEN_POSITION
            OPEN_POSITION['avg_price'] = filled_price
            new_qty = abs(OPEN_POSITION['qty'] - data['filledQty'])
            OPEN_POSITION['qty'] = new_qty
            openned_side = CURRENT_ORDER['side'] if new_qty > 0 else None
            OPEN_POSITION['side'] = openned_side

            # update CURRENT_ORDER
            CURRENT_ORDER['status'] = 'FILLED'

            print('----------AFTER FILLED----------------')
            print('OPEN_POSITION', OPEN_POSITION)
            print('CURRENT_ORDER', CURRENT_ORDER)
            print('--------------------------------------')

            # check state and open profit position
            if openned_side is not None:
                open_position(
                    get_reverse_side(openned_side),
                    'LO',
                    get_profit_price(openned_side, filled_price),
                    False
                )
        if data['cancelQty'] > 0 and request_id not in HANDLED_REQUESTS:
            HANDLED_REQUESTS.append(request_id) # avoid same order events are returned

            request_side = CURRENT_ORDER['side']
            # Update CURRENT_ORDER
            CURRENT_ORDER['status'] = None
            CURRENT_ORDER['side'] = None
            
            # After cancel "profit" position -> create a new "cut loss" position
            if OPEN_ATC:
                open_position(request_side, 'ATC', 0, False)
            else:
                price = CEILING_PRICE if request_side == 'BUY' else FLOOR_PRICE
                open_position(request_side, 'LO', price, False)
        update_redis_db()

In [ ]:
%run data.ipynb
%run db.ipynb

Using redis pub/sub to subscribe ticks

In [ ]:
r = redis.init_redis()
pubsub = r.pubsub()
F1 =  redis.get_key('F1M_CODE').decode()
pubsub.subscribe('HNXDS:{}'.format(F1))

In [ ]:
def init_local_db(redis_data):
    if redis_data is not None:
        global CURRENT_ORDER, OPEN_POSITION, TOTAL_PROFIT
        redis_data_value = ujson.loads(redis_data)
        print(redis_data_value)
        CURRENT_ORDER = redis_data_value['CURRENT_ORDER']
        OPEN_POSITION = redis_data_value['OPEN_POSITION']
        TOTAL_PROFIT = redis_data_value['TOTAL_PROFIT']

In [ ]:
%run config.ipynb

if CUT_LOSS_THRESHOLD >= 0 or TAKE_PROFIT_THRESHOLD <= 0:
    raise Exception('Please check configuration - cut loss must be less than 0 and take profit must be greater than 0')

if CONSUMER_ID == "" or CONSUMER_SECRET == "" or ACCOUNT == "" or PRIVATE_KEY == "":
    raise Exception('Please check configuration - account configuration can not be empty')
    
if START_TRADING_TIME < '09:00:00' or START_TRADING_TIME > '14:30:00':
    raise Exception('Please check configuration - start trading time must be between 09AM and 02:30PM')
    
ssi_api.init_config(
  CONSUMER_ID,
  CONSUMER_SECRET,
  ACCOUNT,
  PRIVATE_KEY
)
token = ssi_api.login()
ssi_connection = ssi_stream.init_stream(token, handle_order_ok)

ACCOUNT_KEY = 'SSI_{}'.format(ACCOUNT)
redis_data = redis.get_key(ACCOUNT_KEY)
init_local_db(redis_data)

In [ ]:
def update_redis_db():
    redis.set_key(ACCOUNT_KEY, ujson.dumps(dict({
        'CURRENT_ORDER': CURRENT_ORDER,
        'OPEN_POSITION': OPEN_POSITION,
        'TOTAL_PROFIT': TOTAL_PROFIT
    })))

Initialize data with redis value (updated by algotrade service)

In [ ]:
all_f1_data = redis.get_key(ALL_F1M_PRICE_TICKS)
if all_f1_data is not None:
    init_ticks(ujson.loads(all_f1_data))

Callback function is used to update `CURRENT_ORDER`

In [ ]:
def update_current_order(data: dict):
    global CURRENT_ORDER
    CURRENT_ORDER = data
    update_redis_db()

This is wrapper function - used to open new position

In [ ]:
def open_position(side: str, order_type: str, price: float, reverse: bool):
    current_side = OPEN_POSITION['side']
    current_order_status = CURRENT_ORDER['status']
    handler.open_position(
        ACCOUNT,
        F1,
        side,
        order_type,
        price,
        reverse,
        MAX_ROUND,
        current_side,
        current_order_status,
        update_current_order
    )

In [ ]:
def cancel_profit_position():
    if CURRENT_ORDER['status'] == 'PENDING_NEW':
        handler.cancel_position(
            ACCOUNT,
            F1,
            CURRENT_ORDER['order_id'],
            CURRENT_ORDER['side'],
            MAX_ROUND,
            update_current_order
        )

Because we want to open position with latest price (ASAP), so we need to pass ceiling price if we buy (open LONG), otherwise, pass floor price if we sell (open SHORT)

In [ ]:
def handle_position_with_price(side: str, order_type: str, data: dict, reverse: bool):
    price = CEILING_PRICE if side == 'BUY' else FLOOR_PRICE
    open_position(side, order_type, price, reverse)

In [ ]:
def get_reverse_side(side: str):
    return 'BUY' if side == 'SELL' else 'SELL'

In [ ]:
def get_profit_price(side: str, price: float):
    pone = 1 if side == 'BUY' else -1
    return price + pone * TAKE_PROFIT_THRESHOLD

In [ ]:
def get_sma_value(value):
    return 'N/A' if value == 0 else value

### Algorithm implementation
Each tick has `last_px` (last price) field, we will add into price list and calculate SMA(t), SMA(t-1)  
About SMA (Simple Moving Average): https://www.investopedia.com/terms/s/sma.asp   

- If last_px(t-1) < SMA(t-1) and last_px(t) >= SMA(t) -> Open **Long**
- If last_px(t-1) > SMA(t-1) and last_px(t) <= SMA(t) -> Open **Short**

If unrealized profit/loss exceed our range (is configured in config.ipynb), we will close position to take profit or cut loss (reverse position)   
We will close openned position in ATC session if we have

In [ ]:
def handle_msg_internal(hidden_info: dict):
    last_px = hidden_info['LastPrice']
    trade_time = hidden_info['Time']
    global OPEN_POSITION, START_TRADING_TIME, FLOOR_PRICE, CEILING_PRICE
    if last_px is not None:
        [prev_last_px, prev_sma, sma] = add_tick(last_px)
        print('SMA(t-1): {}, SMA(t): {}, LAST_PX(t-1): {}, LAST_PX(t): {}'.format(get_sma_value(prev_sma), get_sma_value(sma), prev_last_px, last_px))
        if FLOOR_PRICE is None or CEILING_PRICE is None:
            FLOOR_PRICE = hidden_info['Floor']
            CEILING_PRICE = hidden_info['Ceiling']
                    
        if trade_time >= START_TRADING_TIME and trade_time < '14:30:00':
            if OPEN_POSITION['side'] is not None:
                unrealized = calculate_profit(last_px)
                print('Unrealized Profit/Loss: ', calculate_profit(last_px))
                if unrealized <= CUT_LOSS_THRESHOLD:
                    # cut loss -> close opening position
                    print('Cut loss -> Cancel "profit" position first')
                    cancel_profit_position()
            elif prev_sma > 0.0:
                if prev_last_px < prev_sma and last_px >= sma:
                    print('Long Signal')
                    handle_position_with_price('BUY', 'LO', hidden_info, False)
                if prev_last_px > prev_sma and last_px <= sma:
                    print('Short Signal')
                    handle_position_with_price('SELL', 'LO', hidden_info, False)

    if trade_time > '14:30:00' and OPEN_POSITION['side'] is not None:
        # close opening position in ATC session
        print('Close ATC')
        global OPEN_ATC
        OPEN_ATC = True
        cancel_profit_position()
        # handle_position_with_price(get_reverse_side(OPEN_POSITION['side']), 'ATC', hidden_info, False)

In [ ]:
def handle_msg(msg):
    msg_data = ujson.loads(msg['data'])
    if msg_data['hidden_system_status'] is not None:
        hidden_info = ujson.loads(msg_data['hidden_system_status'])
        handle_msg_internal(hidden_info)
        
        # draw_chart()

### Live logging start here &darr;

In [ ]:
with ssi_connection:
    ssi_connection.start()
    
    for message in pubsub.listen():
        if message['type'] == 'message':
            handle_msg(message)
        if ssi_connection.is_reset_connection:
            print('Lost connection')
            ssi_connection.reset_session(session)
            ssi_connection.start()
            ssi_connection.is_reset_connection = False